# Importing Necessary Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2, imghdr
import numpy as np


# Loading The Data

## Data is loaded using keras can be seen below-

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('/kaggle/input/cat-and-dog-images-dataset/Dog and Cat .png')

In [ ]:
data_iterator = data.as_numpy_iterator()

## This gets a new batch from iterator and can be run again to get new batches

In [ ]:
batch = data_iterator.next()

In [ ]:
len(batch)

In [ ]:
batch[0].shape

In [ ]:
batch[1]

# Visualising The Data(Using matplotlib library)

In [ ]:
fig, ax = plt.subplots(ncols = 5, figsize = (20,20))
for idx, img in enumerate (batch[0][:5]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

## As we can see above that
### Number '0' --> Cats
### Number '1' --> Dogs

In [ ]:
batch[0].max()

In [ ]:
scaled = batch[0]/255
scaled.max()

# Preprocessing The Data

### So,as we have seen that it is very important to scale the data and doing that one by one is not feasable for bigger data so to do that easily we scale the data in a pipeline

In [ ]:
data = data.map(lambda x, y : (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
batch = scaled_iterator.next()

# Re-Visualising The Data

In [ ]:
fig, ax = plt.subplots(ncols = 5, figsize = (20,20))
for idx, img in enumerate (batch[0][:5]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

# Splitting The Data(For Training, Testing , Validating)

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)

train_size + val_size +test_size

In [ ]:
train = data.take(train_size)
valid = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

# Making Deep Learning Model

### Building Neural Networks

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense

In [ ]:
model = Sequential()

### The Conv2D layer is defined by parts below-

In [ ]:
# model.add(Conv2D(<no of filters to be done on image>, <on what shape the filters are done>, <stride>, <activation>, <input_shape>))
model.add(Conv2D(16, (3,3), 1, activation = 'relu', input_shape = (256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation = 'relu'))

model.add(Dense(1, activation = 'sigmoid'))

### Here,The Optimizer We Use Is ADAM and losses are measured by BinaryCrossEntropy at last the metrics is set which is Accuracy

In [ ]:
model.compile('adam', loss = tf.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [ ]:
model.summary()

## Training The Network

In [ ]:
logdir = 'logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [ ]:
history = model.fit(train, epochs = 20, validation_data = valid, callbacks = [tensorboard_callback] )

## Plotting The Performance Of Model

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color = 'purple', label = 'loss')
plt.plot(history.history['val_loss'], color = 'red', label = 'val_loss')
fig.suptitle('Loss', fontsize = 20)
plt.legend(loc = "upper left")
plt.show()

fig = plt.figure()
plt.plot(history.history['accuracy'], color = 'purple', label = 'accuracy')
plt.plot(history.history['val_accuracy'], color = 'red', label = 'val_accuracy')
fig.suptitle('Accuracy', fontsize = 20)
plt.legend(loc = "upper left")
plt.show()

# Evaluating Performance

In [ ]:
from tensorflow.keras.metrics import Precision , Recall , BinaryAccuracy

In [ ]:
precision = Precision()
recall = Recall()
accuracy = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    accuracy.update_state (y, yhat)

In [ ]:
print(f'Precision:{precision.result().numpy()}, Recall:{recall.result().numpy()}, Accuracy:{accuracy.result().numpy()}')

# Testing

In [ ]:
import cv2

In [ ]:
img = cv2.imread('/kaggle/input/cat-test-jpg/cat_61.jpg' )
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

### Now We know our model accept the image in size 256 * 256,so we will resize the test image

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5:
    print("Predicted Outcome = DOG")
else:
    print("Predicted Outcome = CAT")

# Saving The Model

In [ ]:
import os

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models', 'ImageClassifierCatDog.h5 '))

In [ ]:
new_model = load_model(os.path.join('models', 'ImageClassifierCatDog.h5 ') )

In [ ]:
yhat_new = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
if yhat_new > 0.5:
    print("Predicted Outcome = DOG")
else:
    print("Predicted Outcome = CAT")